## Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.0.0-beta1
2.2.4-tf


## Enable Eager Mode

In [2]:
#tf.enable_eager_execution()

## Hyper Parameters

In [3]:
learning_rate = 0.001
training_epochs = 25
batch_size = 100
n_class = 10

## MNIST/Fashion MNIST Data

In [4]:
## MNIST Dataset #########################################################
mnist = keras.datasets.mnist
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
##########################################################################

## Fashion MNIST Dataset #################################################
#mnist = keras.datasets.fashion_mnist
#class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
##########################################################################

## Datasets

In [5]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()  

In [6]:
n_train = train_images.shape[0]
n_test = test_images.shape[0]

In [7]:
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
    
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)    
    
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(
                buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

## Model Function

In [8]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    model.add(keras.layers.Dense(256,
                                activation='relu'))
    model.add(keras.layers.Dense(128,
                                activation='relu'))
    model.add(keras.layers.Dense(10,
                                activation='softmax'))
    return model

In [9]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
_________________________________________________________________


## Loss Function

In [10]:
@tf.function
def loss_fn(model, images, labels):
    predictions = model(images, training=True)
    loss = tf.reduce_mean(keras.losses.categorical_crossentropy(labels, predictions))   
    return loss  

## Calculating Gradient & Updating Weights

In [11]:
@tf.function
def train(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

## Caculating Model's Accuracy

In [12]:
@tf.function
def evaluate(model, images, labels):
    predictions = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))    
    return accuracy

## Optimizer

In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

## Training

In [14]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_dataset:
        train(model,images, labels)
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(model, images, labels)        
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))


print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 1 loss = 0.22955699 train accuracy =  0.9367 test accuracy =  0.9618
Epoch: 2 loss = 0.08245207 train accuracy =  0.9758 test accuracy =  0.9650
Epoch: 3 loss = 0.04984841 train accuracy =  0.9860 test accuracy =  0.9735
Epoch: 4 loss = 0.03166842 train accuracy =  0.9920 test accuracy =  0.9735
Epoch: 5 loss = 0.02090867 train accuracy =  0.9955 test accuracy =  0.9736
Epoch: 6 loss = 0.01475032 train accuracy =  0.9970 test accuracy =  0.9781
Epoch: 7 loss = 0.01127579 train accuracy =  0.9978 test accuracy =  0.9771
Epoch: 8 loss = 0.00906437 train accuracy =  0.9983 test accuracy =  0.9787
Epoch: 9 loss = 0.00730594 train accuracy =  0.9986 test accuracy =  0.9795
Epoch: 10 loss = 0.00524292 train accuracy =  0.9991 test accuracy =  0.9758
Epoch: 11 loss = 0.00449659 train accuracy =  0.9993 test accuracy =  0.9789
Epoch: 12 loss = 0.00430536 train accuracy =  0.9994 test accuracy =  0.9774
Epoch: 13 loss = 0.00475246 train accuracy =  0.